In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Conv1D, Dense, Dropout, Flatten
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.optimizers import Adam


In [3]:
# Load the Bid and Ask datasets
bid_data = pd.read_csv('XAGUSD_5 Mins_Bid_2023.01.01_2023.06.30.csv')  # Replace with your bid file path
ask_data = pd.read_csv('XAGUSD_5 Mins_Ask_2023.01.01_2023.06.30.csv')  # Replace with your ask file path


In [4]:
# Merge datasets on 'Time (UTC)'
data = pd.merge(bid_data, ask_data, on='Time (UTC)', suffixes=('_Bid', '_Ask'))

In [ ]:
# Plot Open prices and Volume for visualization
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(data['Time (UTC)'], data['Open_Bid'], label='Open_Bid')
plt.plot(data['Time (UTC)'], data['Open_Ask'], label='Open_Ask')
plt.legend()
plt.title("Open Prices (Bid and Ask)")

Text(0.5, 1.0, 'Open Prices (Bid and Ask)')

Error in callback <function _draw_all_if_interactive at 0x000002556FD66FC0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

In [ ]:

plt.subplot(2, 1, 2)
plt.plot(data['Time (UTC)'], data['Volume_Bid'], label='Volume_Bid')
plt.plot(data['Time (UTC)'], data['Volume_Ask'], label='Volume_Ask')
plt.legend()
plt.title("Volume (Bid and Ask)")
plt.xlabel('Time')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_data = scaler.fit_transform(data[['Open_Bid', 'High_Bid', 'Low_Bid', 'Close_Bid', 'Volume_Bid',
                                             'Open_Ask', 'High_Ask', 'Low_Ask', 'Close_Ask', 'Volume_Ask']])

In [ ]:
# Shift min and max prices
data_shifted = pd.DataFrame(normalized_data, columns=['Open_Bid', 'High_Bid', 'Low_Bid', 'Close_Bid', 'Volume_Bid',
                                                      'Open_Ask', 'High_Ask', 'Low_Ask', 'Close_Ask', 'Volume_Ask'])
data_shifted['Shifted_Min'] = data_shifted[['Low_Bid', 'Low_Ask']].min(axis=1)
data_shifted['Shifted_Max'] = data_shifted[['High_Bid', 'High_Ask']].max(axis=1)

In [ ]:
# Separate inputs (X) and outputs (y)
n_steps = 60  # Define the sequence length
X = []
y = []
for i in range(n_steps, len(data_shifted)):
    X.append(data_shifted.iloc[i-n_steps:i].values)  # Input sequence
    y.append(data_shifted['Close_Bid'].iloc[i])  # Target is the Bid close price

X = np.array(X)
y = np.array(y)

In [ ]:
# Split into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [ ]:
# Define the LSTM Model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mse')

In [ ]:

# Train the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


In [ ]:
# Predict with the LSTM model
lstm_predictions = lstm_model.predict(X_test)
lstm_predictions = scaler.inverse_transform(np.concatenate((lstm_predictions, np.zeros((lstm_predictions.shape[0], X.shape[2]-1))), axis=1))[:, 0]


In [ ]:

# CNN model
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])
cnn_model.compile(optimizer=Adam(), loss='mse')

In [ ]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)


In [ ]:
# Predict with the CNN model
cnn_predictions = cnn_model.predict(X_test)
cnn_predictions = scaler.inverse_transform(np.concatenate((cnn_predictions, np.zeros((cnn_predictions.shape[0], X.shape[2]-1))), axis=1))[:, 0]


In [ ]:
# Calculate MSE and MAE for both models
lstm_mse = mean_squared_error(y_test, lstm_predictions)
lstm_mae = mean_absolute_error(y_test, lstm_predictions)

cnn_mse = mean_squared_error(y_test, cnn_predictions)
cnn_mae = mean_absolute_error(y_test, cnn_predictions)

print(f"LSTM Model - MSE: {lstm_mse}, MAE: {lstm_mae}")
print(f"CNN Model - MSE: {cnn_mse}, MAE: {cnn_mae}")


In [ ]:
# Plotting the Results
plt.figure(figsize=(12, 6))
plt.plot(scaler.inverse_transform(y_test.reshape(-1, 1)), label='True Prices')
plt.plot(lstm_predictions, label='LSTM Predictions')
plt.plot(cnn_predictions, label='CNN Predictions')
plt.legend()
plt.title("Price Predictions Comparison")
plt.xlabel("Time")
plt.ylabel("Price")
plt.show()